In [1]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
import tensorflow as tf

# tf.get_logger().setLevel(logging.ERROR)

# Allow memory growth for the GPU
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Activate mixed precision
from keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")
from dino import create_dino_model, Sequential, layers, ResNet

2023-11-22 02:04:50.535574: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 02:04:51.263165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Ti, compute capability 8.6
Using TensorFlow backend


2023-11-22 02:04:52.429643: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:04:52.532653: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:04:52.532708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:04:52.533129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
BATCH_SIZE = 2

In [3]:
# All images under this path will be used for training
dataset_path = "/mnt/c/Users/mnusr/PycharmProjects/hcc_tests/big_image_example/processed_2"

# All images under this path will be used for testing
test_dataset_path = "/mnt/c/Users/mnusr/PycharmProjects/hcc_tests/datasets/hcc"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    label_mode=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(2048, 2048),
    shuffle=True,
    seed=42,
    interpolation="bicubic",
).prefetch(tf.data.AUTOTUNE)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    label_mode='binary',
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(1024, 1024),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
    interpolation="bicubic",
).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    label_mode='binary',
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(1024, 1024),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
    interpolation="bicubic",
).prefetch(tf.data.AUTOTUNE)



Found 38992 files belonging to 1 classes.


2023-11-22 02:07:03.642549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:07:03.642620: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:07:03.642651: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:07:03.913744: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-22 02:07:03.913833: I tensorflow/compile

Found 1670 files belonging to 2 classes.
Using 1336 files for training.
Found 1670 files belonging to 2 classes.
Using 334 files for validation.


In [4]:
# Zero label count
total_zero_labels = 0
total_one_labels = 0
for batch in validation_dataset:
    total_zero_labels += tf.math.count_nonzero(batch[1] == 0)
    total_one_labels += tf.math.count_nonzero(batch[1] == 1)

print(f"Total zero labels: {total_zero_labels}")
print(f"Total one labels: {total_one_labels}")
print(f"Total labels: {total_zero_labels + total_one_labels}")
class_weight = {0: total_one_labels / (total_zero_labels + total_one_labels),
                1: total_zero_labels / (total_zero_labels + total_one_labels)}
print(f"Class weight: {class_weight}")

Total zero labels: 866
Total one labels: 470
Total labels: 1336
Class weight: {0: <tf.Tensor: shape=(), dtype=float64, numpy=0.35179640718562877>, 1: <tf.Tensor: shape=(), dtype=float64, numpy=0.6482035928143712>}


In [5]:
teacher = ResNet(
    rescale=True,
    input_shape=(None, None, 3),
    batch_count=BATCH_SIZE,
    activations="gelu",
)

student = ResNet(
    rescale=True,
    input_shape=(None, None, 3),
    batch_count=BATCH_SIZE,
    activations="gelu",
)

model = create_dino_model(
    student,
    teacher,
    global_crop_size=512,
    local_crop_size=256,
    local_augmentation_count=4,
    network_momentum=0.9995,
    teacher_temperature=0.04
)

schedule = tf.keras.optimizers.schedules.CosineDecay(0.00001, decay_steps=152 * 4, alpha=1e-6, warmup_target=0.0005,warmup_steps=152 * 10)

optimizer = tf.keras.optimizers.Adam(schedule)

scaled_optimizer = mixed_precision.LossScaleOptimizer(optimizer)

model.compile(optimizer=scaled_optimizer, run_eagerly=False, jit_compile=False)



In [6]:
student.trainable = True
model.fit(
    train_dataset,
    epochs=8
)

Epoch 1/8
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow


2023-11-22 02:08:35.286892: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-11-22 02:08:36.780923: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-22 02:08:37.050211: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 597.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-22 02:08:37.073718: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.15GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-22 02:08:37.087818: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU

  127/19496 [..............................] - ETA: 2:56:05 - loss: 110.9183Gradients applied, step: <tf.Variable 'iteration:0' shape=() dtype=int64>


2023-11-22 02:09:51.039182: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbd26aee9a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-22 02:09:51.039216: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2023-11-22 02:09:51.043774: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-22 02:09:51.154482: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19496/19496 [==============================] - 10447s 532ms/step - loss: 113.0256
Epoch 2/8
19496/19496 [==============================] - 10336s 530ms/step - loss: 112.8985
Epoch 3/8
19496/19496 [==============================] - 10401s 533ms/step - loss: 109.7490
Epoch 4/8
19496/19496 [==============================] - 10352s 531ms/step - loss: 109.7392
Epoch 5/8
19496/19496 [==============================] - 10321s 529ms/step - loss: 109.7430
Epoch 6/8
19496/19496 [==============================] - 10368s 532ms/step - loss: 118.3578
Epoch 7/8
14710/19496 [=====================>........] - ETA: 42:29 - loss: 123.3054

KeyboardInterrupt: 

In [13]:
student.trainable = False

model_st = Sequential(
    [
        student,
        layers.Dense(1),
        layers.Activation("sigmoid", dtype=tf.float32)
    ])

optimizer_st = tf.keras.optimizers.Adam(0.0000001)

scaled_optimizer_st = mixed_precision.LossScaleOptimizer(optimizer_st)

model_st.compile(optimizer=scaled_optimizer_st, run_eagerly=False, jit_compile=True,
                 metrics=[
                     tf.metrics.BinaryAccuracy(name='accuracy'),
                     tf.metrics.AUC(name='auc'),
                     tf.metrics.Precision(name='precision'),
                     tf.metrics.Recall(name='recall'),
                     tf.metrics.F1Score(name='f1')
                 ],
                 loss="binary_crossentropy")

In [14]:
model_st.fit(
    validation_dataset,
    validation_data=test_dataset,
    epochs=20,
    initial_epoch=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
    ,
    class_weight= class_weight
)

Epoch 1/20
668/668 [==============================] - 32s 36ms/step - loss: 1.8333 - accuracy: 0.3518 - auc: 0.5036 - precision: 0.3518 - recall: 1.0000 - f1: 0.5205 - val_loss: 4.1516 - val_accuracy: 0.4102 - val_auc: 0.5140 - val_precision: 0.4102 - val_recall: 1.0000 - val_f1: 0.5817
Epoch 2/20
668/668 [==============================] - 23s 33ms/step - loss: 1.3895 - accuracy: 0.3518 - auc: 0.5682 - precision: 0.3518 - recall: 1.0000 - f1: 0.5205 - val_loss: 3.0072 - val_accuracy: 0.4102 - val_auc: 0.6691 - val_precision: 0.4102 - val_recall: 1.0000 - val_f1: 0.5817
Epoch 3/20
668/668 [==============================] - 23s 34ms/step - loss: 0.9614 - accuracy: 0.3518 - auc: 0.6179 - precision: 0.3518 - recall: 1.0000 - f1: 0.5205 - val_loss: 1.9301 - val_accuracy: 0.4102 - val_auc: 0.6703 - val_precision: 0.4102 - val_recall: 1.0000 - val_f1: 0.5817
Epoch 4/20
668/668 [==============================] - 23s 34ms/step - loss: 0.5949 - accuracy: 0.3555 - auc: 0.6159 - precision: 0.3529 

In [ ]:
model.save('dino')
model_st.save('dino_student')

In [ ]:
# 0.5817